**Import necessary library**

In [10]:
import requests
import certifi
import json
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import FixedLocator
from IPython.display import display, HTML
import numpy as np

**Replace with your API key**

In [11]:
api_key = "255446bcdde7ca9fe776258d09e8411bbb8d1cade2ebd6aba440f80f6817c3fd"

**EX3. Frequencies of HAS and HATH in the plays**

**Text set = Shakespeare 20 for demos**  
**Segment by text**  
**Include metadata in output**  
**Words unsorted – HAS HATH**  
**Output – choose proportions**  
**Transform and chart – sort by DATE, plot HAS and HATH as lines**

In [ ]:
# 3. Frequencies of HAS and HATH in the plays 

# Text set = Shakespeare 20 for demos 
# Segment by text 
# Include metadata in output 
# Words unsorted – HAS HATH 
# Output – choose proportions 
# Transform and chart – sort by DATE, plot HAS and HATH as lines 

# Replace with your actual text set ID 
textset_id = 86 

#Character to exlude from search
excludeWords = ["[","\\", "]", "_", "`", "!", "\"", "#", "%", "'", "(", ")", "+", ",", "-", "–", ".", "/", ":", ";", "{", "|", "}", "=", "~", "?" ]

request_url = "https://sia.ardc-hdcl-sia-iaw.cloud.edu.au/api/v1/word-frequencies"
character_parts_request = {
    'textSet': textset_id,
    'option': {
        'blockMethod' : 0,         #Segment by text
        'showMetadata' : True,
        'outputSize': 1000 , 
        # 'outputSpecialWords' : ["has","hath"],
        # 'outputSpecialWordsOption' : 0,
        'excludeWords': excludeWords,
    }
}


# Make API request
response = requests.post(request_url, json=character_parts_request, headers={"X-API-KEY": api_key}, timeout=1200)

special_word_count_by_year = {"has": {}, "hath": {}}
total_word_count_by_year = {}

# Handle the response
if response.status_code == 200:
    response_data = response.json()
    blocks = response_data.get("blocks", [])
    

    for block in blocks:
        name = block.get("name", "")
        freqs = block.get("frequencies", [])
        
        # Extract year from name
        year = block.get('metadata').get('Date_of_publication')
        
        name.split("_")[-1].split(" ")[0]
        
        for freq in freqs:
            word = freq.get("word", "")
            value = freq.get("value", 0)
            
            if word.lower() == "has":
                special_word_count_by_year["has"][year] = special_word_count_by_year["has"].get(year, 0) + value
            elif word.lower() == "hath":
                special_word_count_by_year["hath"][year] = special_word_count_by_year["hath"].get(year, 0) + value
                
            total_word_count_by_year[year] = total_word_count_by_year.get(year, 0) + value

    # Sort by Year
    sorted_years = sorted(set(special_word_count_by_year["has"].keys()) | set(special_word_count_by_year["hath"].keys()))
    has_counts = [(special_word_count_by_year["has"].get(year, 0) / total_word_count_by_year.get(year, 1)) * 100 for year in sorted_years]
    hath_counts = [(special_word_count_by_year["hath"].get(year, 0) / total_word_count_by_year.get(year, 1)) * 100 for year in sorted_years]

    plt.figure(figsize=(12, 6))
    plt.plot(sorted_years, has_counts, label='HAS', marker='o')
    plt.plot(sorted_years, hath_counts, label='HATH', marker='o')
    plt.scatter(sorted_years, has_counts)
    plt.scatter(sorted_years, hath_counts)
    plt.xlabel('Year')
    plt.ylabel('Percentage')
    plt.title('Percentage of "HAS" and "HATH" over years')

    y_ticks = plt.gca().get_yticks()
    plt.gca().yaxis.set_major_locator(FixedLocator(y_ticks))
    plt.gca().set_yticklabels(['{:.2f}%'.format(y) for y in y_ticks])

    plt.legend()
    plt.show()
    
    
    # Metadata
    html = '<h2>Metadata</h2>'
    html += '<div style="overflow: auto; max-height: 500px; margin-top: 40px;">'
    html += '<table border="1">'
     # Adding the table header for Metadata
    if blocks:
        first_metadata = blocks[0].get("metadata", {})
        html += '<tr>'
        for header in first_metadata.keys():
            html += f'<th style="white-space: nowrap;">{header}</th>'
        html += '</tr>'
        
        # Adding Metadata data rows
        for block in blocks:
            metadata = block.get("metadata", {})
            html += '<tr>'
            for key in first_metadata.keys():  # Ensure the same order as headers
                html += f'<td style="white-space: nowrap;">{metadata.get(key, "N/A")}</td>'
            html += '</tr>'
        
    html += '</table></div>'
    
    # Display the table
    display(HTML(html))

    
else:
    print(f"Failed: {response.status_code} {response.reason}")